In [1]:
!pip install ../pydensecrf_package/pydensecrf-1.0rc2-cp39-cp39-win_amd64.whl

Processing c:\users\fcq20zl\desktop\crf_method\pydensecrf_package\pydensecrf-1.0rc2-cp39-cp39-win_amd64.whl
pydensecrf is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [2]:
import numpy as np
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral
import numpy as np
import matplotlib.pyplot as plt
import math
import torch 
import torch.nn.functional as F
import glob
import cv2
from PIL import Image
import os
%matplotlib inline

plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
from scipy.stats import multivariate_normal



# 
def softmax(x,axis=None):
    x = x - x.max(axis=axis, keepdims=True)
    y = np.exp(x)
    y = y / y.sum(axis=axis, keepdims=True)
    return y

In [7]:
para_1 = 10  # location
para_2 =  10  # image content
para_3 =  10 # weight
num_iter = 10;

loo_xx = 'loo3';

#################################################################################################################  
# read the raw MRI 
MRI_path = glob.glob('../save_to_npy/Dataset/NW/'+MB_XX+'/MRI/*.png')
MRI_path = sorted(MRI_path)
# read the npy saved from the end of the nerual network
pred_path = glob.glob('../save_to_npy/'+MB_XX+'_npy/'+MB_XX+'_npy_'+loo_xx+'/*.npy')
pred_path = sorted(pred_path)
print(len(MRI_path))
print(len(pred_path))
print('..................')

for nn in range(0,len(MRI_path)):
    img = cv2.imread(MRI_path[nn],cv2.IMREAD_GRAYSCALE); # 
    output = np.load(pred_path[nn]); # the dim of pred is 256 256 26 
    
    output = softmax(output,axis=0);#
    
    
    # Inference without pair-wise terms
    U = unary_from_softmax(output)  # note: num classes is first dim
    W, H, NLABELS = 256,256,26
    d = dcrf.DenseCRF2D(W, H, NLABELS)
    d.setUnaryEnergy(U)
    
    # Create the pairwise bilateral term from the above image.
    # The two `s{dims,chan}` parameters are model hyper-parameters defining
    # the strength of the location and image content bilaterals, respectively.
    pairwise_energy = create_pairwise_bilateral(sdims=(para_1,), schan=(para_2,), img=img, chdim=-1)
    d.addPairwiseEnergy(pairwise_energy, compat=para_3)  # `compat` is the "strength" of this potential.
    d.addPairwiseGaussian(sxy=1, compat=1)
    
    Q, tmp1, tmp2 = d.startInference()
    
  
    
    for _ in range(num_iter):
        d.stepInference(Q, tmp1, tmp2)
    map_soln3 = np.argmax(Q, axis=0).reshape((H,W))
    
    map_soln3.astype(np.uint8)
    if (nn+1) < 10:
          filename = './.........png'
          cv2.imwrite(filename, map_soln3)
    elif (nn+1)>=10 and (nn+1)<=99:
          filename = './.........png'
          cv2.imwrite(filename, map_soln3)
    else:
          filename = './.........png'
          cv2.imwrite(filename, map_soln3)
        
        
        
    

555
555
..................
